In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [2]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/"

if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)
    
### IS THIS RACHAEL'S DATASET OR SOMEONE WHO HAS VIDEOINFO.TXT IN THEIR FOLDER???

all_folders_drop, excel_drop, txt_drop = get_dropbox_video_folders("/DATA/MYRISTATE/DATA/1_week/", True)

# folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "452"]
# all_folders_drop = pd.read_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

clear_output(wait=False)

all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230301_Plate016_01/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,01/Img/
1,20230301_Plate016_02/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,02/Img/
2,20230301_Plate016_03/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,03/Img/
3,20230301_Plate016_05/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,05/Img/
4,20230301_Plate016_06/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,06/Img/
...,...,...,...,...,...
302,20230515_Plate059_018/Img/,059,20230515,DATA/MYRISTATE/DATA/1_week/20230515_Plate059/0...,018/Img/
303,20230515_Plate059_019/Img/,059,20230515,DATA/MYRISTATE/DATA/1_week/20230515_Plate059/0...,019/Img/
304,20230515_Plate059_020/Img/,059,20230515,DATA/MYRISTATE/DATA/1_week/20230515_Plate059/0...,020/Img/
305,20230515_Plate059_021/Img/,059,20230515,DATA/MYRISTATE/DATA/1_week/20230515_Plate059/0...,021/Img/


In [11]:
# directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_dataset/"
# all_folders_drop.to_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# save_dropbox_state("/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate552/", is_video=True)

folders_drop = all_folders_drop
# folders_drop = folders_drop.loc[folders_drop["video"] == "084"]

In [12]:
folders_drop = folders_drop.sort_values(by=['folder'], ignore_index=True)
folders_drop["folder"] = [f"{entry.split('_')[0]}_{entry.split('_')[1]}/{entry.split('_')[2]}" for entry in folders_drop["folder"]]

In [13]:
# FINAL CHECK

print(directory_targ)
folders_drop

/gpfs/scratch1/shared/amftrackflow/Hannah_set/1_week/


,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230301_Plate016/01/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,01/Img/
1,20230301_Plate016/02/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,02/Img/
2,20230301_Plate016/03/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,03/Img/
3,20230301_Plate016/04/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,04/Img/
4,20230301_Plate016/05/Img/,016,20230301,DATA/MYRISTATE/DATA/1_week/20230301_Plate016/0...,05/Img/
...,...,...,...,...,...
302,20230518_Plate063/011/Img/,063,20230518,DATA/MYRISTATE/DATA/1_week/20230518_Plate063/0...,011/Img/
303,20230518_Plate063/012/Img/,063,20230518,DATA/MYRISTATE/DATA/1_week/20230518_Plate063/0...,012/Img/
304,20230518_Plate063/013/Img/,063,20230518,DATA/MYRISTATE/DATA/1_week/20230518_Plate063/0...,013/Img/
305,20230518_Plate063/014/Img/,063,20230518,DATA/MYRISTATE/DATA/1_week/20230518_Plate063/0...,014/Img/


In [14]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "10:00:00",
    "transfer_test"
)


Submitted batch job 2872856


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2872857


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2872858


...

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2874203


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2874204
Submitted batch job 2874205


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


In [16]:
print(excel_drop)
for xl_adress in excel_drop:
    file_name = f"{xl_adress.split('/')[-2]}/{xl_adress.split('/')[-1]}"
    
    download(xl_adress, directory_targ+file_name)

['/DATA/MYRISTATE/DATA/1_week/20230301_Plate016/20230301_plate016.csv', '/DATA/MYRISTATE/DATA/1_week/20230307_Plate010/20230307_plate010.csv', '/DATA/MYRISTATE/DATA/1_week/20230307_Plate030/20230307_plate030.csv', '/DATA/MYRISTATE/DATA/1_week/20230313_Plate026/20230313_plate026.csv', '/DATA/MYRISTATE/DATA/1_week/20230301_Plate046/20230301_plate046.csv', '/DATA/MYRISTATE/DATA/1_week/20230308_Plate048/20230308_plate048.csv', '/DATA/MYRISTATE/DATA/1_week/20230508_Plate067/20230508_Plate067.xlsx', '/DATA/MYRISTATE/DATA/1_week/20230508_Plate068/20230508_Plate068.xlsx', '/DATA/MYRISTATE/DATA/1_week/20230508_Plate069/20230508_Plate069.csv', '/DATA/MYRISTATE/DATA/1_week/20230515_Plate064/20230515_Plate064.xlsx', '/DATA/MYRISTATE/DATA/1_week/20230518_Plate049/20230518_Plate049.csv', '/DATA/MYRISTATE/DATA/1_week/20230518_Plate050/20230518_Plate050.csv', '/DATA/MYRISTATE/DATA/1_week/20230518_Plate051/20230518_Plate051.csv', '/DATA/MYRISTATE/DATA/1_week/20230518_Plate055/20230518_Plate055.csv', '/